In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForMaskedLM, AutoModel, AutoTokenizer, GPT2Tokenizer, AutoModelForCausalLM
import pandas as pd
import numpy as np
from evaluate import load
bertscore = load("bertscore")

## Generating Agent Predictions 

In [105]:
def generate(context, model, tokenizer):
  # create prompt
  prompt = context + " agent: "
  print("prompt: ", prompt)

  # tokenize input (string -> tensor)
  input_ids = tokenizer(prompt, return_tensors='pt').input_ids

  # also save to csv to do a sanity check
  output_ids = model.generate(input_ids, max_length=800, eos_token_id=tokenizer.encode("customer: "))
  
  # decode output (tensor -> string)
  pred_response = tokenizer.batch_decode(output_ids, skip_special_tokens = True)


  # slice the output: output_ids = outputs[:, input_ids.shape[-1]:]
  output_string = pred_response[0]
  prefix = context + " agent: "
  suffix = "customer: "
  # og_output.append(output_string.removeprefix(prefix))
  sliced_output = output_string.removeprefix(prefix)
  # sliced_output = output_string.removeprefix(prefix).rstrip(suffix)


  return sliced_output

In [ ]:
model = AutoModelForCausalLM.from_pretrained("../finetuning/distilgpt2-finetuned-wikitext2/checkpoint-5000").eval()
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [106]:
df = pd.read_csv("evaluation_csv_v1.csv")
# need to do subsample
df_subsample = df.sample(n=25, random_state=1) 
output_df = pd.DataFrame(columns=["context", "true_agent_response", "predicted_response", 'BERT_precision', 'BERT_recall', "BERT_f1"])
all_pred_responses, all_true_responses = [], []

for index, row in df_subsample.iterrows():
    pred_response = generate(row["context"], model, tokenizer)
    all_pred_responses.append(pred_response)
    all_true_responses.append(row["true_agent_response"]) 
    similarity_score = bertscore.compute(predictions=[pred_response], references=[row["true_agent_response"]], lang="en")
    new_row = {"context": row["context"], "true_agent_response":row["true_agent_response"], "predicted_response": pred_response, 'BERT_precision':similarity_score["precision"][0], "BERT_recall": similarity_score["recall"][0], "BERT_f1": similarity_score["f1"][0]}
    output_df = pd.concat([output_df, pd.DataFrame([new_row])], ignore_index=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.


prompt:   agent: Hello how can I help you today? customer: Hi good morning. customer: I just have a quick question agent: sure thing customer: Is it possible to add an item to an existing order? customer: I had forgotten to add it earlier. agent: I can help with that agent: Can I start with your full name or account id? customer: Sanya Afzal agent: The account is now pulled up, now can I have the account id and order id to verify identity. customer: Yes. customer: My account ID is OWGCTKCE0R customer: Order ID: 1950650472 agent: Can you tell me the current shipping status of the order now that the identity verification is complete? customer: The shipping status says "order received." agent: 


/var/folders/ml/t6mthscj76j1mqyxmp4kk9880000gn/T/ipykernel_20159/767821616.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, pd.DataFrame([new_row])], ignore_index=True)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.


prompt:   agent: Hello, how can I help you today? customer: Hi, so... I feel kind of silly. It's been a while since I was on the site and I'm trying to log in customer: but i cant seem to remember my username :( agent: Don't feel silly! That is so common! agent: 
prompt:   agent: how may i help you customer: I'm thinking of buying some things, and I needed to know how to get a premium membership. agent: i see, so you are inquiring about how to get a subscription today? agent: i can help with that, can i get your name please? agent: are you there agent: i have your answer here agent: membership is based on the dollar value of purchases within 1 yyear agent: 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.


...

prompt:   customer: Hi customer: I have an issue with my order customer: Can you help? agent: Hello, how may I help you today? agent: 
prompt:   agent: Hello, how can i help you customer: Hi. I have this promo code but i just can't get it to work. agent: I will be happy to help you with this issue. agent: 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.


prompt:   agent: Hello! Thank you for shopping with AcmeBrands. How may I assist you? customer: hi agent: Hello! agent: How may I help you today? customer: I would like to access my account customer: but I've forgotten my username agent: I'll be glad to help you with that. May I get your full name, please? customer: Rodriguez Domingo agent: Thank you, Rodriguez. To help me verify your identity, would you please provide your zip code, phone number, and email address? Two out of those three options would be fine. customer: 1172 customer: (956) 645-6350 customer: rodriguezdomingo971@email.com agent: Thank you, you're verified! agent: Your username is rdomingo1 and you may use it now to login and complete your purchase. agent: Is there anything more I may help you with today? customer: Thanks customer: That's all agent: 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.


prompt:   customer: Hello, I am Norman Bouchard agent: Hello Norman, Thank you for contacting us today. How may I help you? customer: I think I got charged a different amount for my order customer: It says it was $400 but the item was only $99 customer: Order ID: 4411508179 customer: So not sure if the email is wrong agent: That seems like a big difference i'll do my best to get it sorted. agent: 
prompt:   agent: Hello, thank you for contacting AcmeCorp, how may I help you today? customer: Hello. I wanted to purchase something, but my credit card keeps getting rejected. customer: Would you happen to know why it's considered invalid? agent: Oh no, I would happy to help. agent: Lets try a few options. Can you please check the card to make sure the expiration date is still valid? customer: Yes, the expiration date is still valid. agent: 


In [107]:
output_df.to_excel("output_v3.xlsx")

## Evaluation Using BERT

In [88]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

In [89]:
results

{'precision': [0.9999998807907104, 1.0000001192092896],
 'recall': [0.9999998807907104, 1.0000001192092896],
 'f1': [0.9999998807907104, 1.0000001192092896],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.28.0)'}